# GMAT Impulsive Burn Modeling
This notebook provides a quick introduction to using the GMAT API to fire an Impulsive burn using the GMAT ImpulsiveBurn class. To proceed, we'll need to prepare the GMAT API for use.

In [1]:
# Load the GMAT API and initialize the GMAT engine
from load_gmat import *
# Load NumPy for convenience
import numpy as np

Impulsive burns in GMAT are applied to spacecraft.  When mass depletion is included, the spacecraft also needs a fuel tank, and the burn needs to have that tank set as the source for the mass depletion.  Accordingly, we need 3 objects: a fuel tank, an impulsive burn, and spacecraft object:

In [2]:
tank = gmat.Construct("ChemicalTank","FuelTank")
burn = gmat.Construct("ImpulsiveBurn","Impulse")
sat = gmat.Construct("Spacecraft","Sat")

The default impulsive burn has magnitude zero.  Instead, we'll apply a 100 m/s burn, set in the velocity direction in the following code.  This code also sets the burn to deplete mass, and sets the mass properties for the spacecraft and fuel so that the depletion can be compared to analytic checks.

In [3]:
# Use a 30 kg spacecraft
sat.SetField("DryMass", 30)
# Add the fuel tank to the spacecraft's hardware
sat.SetField("Tanks","FuelTank")

# Use 20 kg of fuel
tank.SetField("FuelMass", 20)

# Finally, set up the burn to deplete mass from the tank for a 100 m/s burn
burn.SetField("DecrementMass",True)
burn.SetField("Tank","FuelTank")
burn.SetField("Element1", 0.1)

The configuration here identifies the object connections, including some default settings like the space environment (that is, the solar system) and the spacecraft state. In GMAT, the underlying objects are connected together when the system is iniitalized.

GMAT's impulsive burns are general purpose objects that can be applied to multiple spacecraft.  When running a GMAT script, the Maneuver command connects the burn to a specific spacecraft.  In the API, that connection needs to be made by passing the spacecraft into the burn object, using the SetRefObject() method.  That connection needs to be made before initialization so that the burn can build the coordinate system objects it needs to apply the maneuver when it fires.  

The following code sets the spacecraft reference and initializes the components for use.

In [4]:
burn.SetRefObject(sat, sat.GetType(), sat.GetName())
gmat.Initialize()

Now that the components are configured, the maneuver can be applied, and its results displayed:

In [5]:
# Initial position and velocity
pos_i = np.array([sat.GetNumber("X"), sat.GetNumber("Y"), sat.GetNumber("Z")])
vel_i = np.array([sat.GetNumber("VX"), sat.GetNumber("VY"), sat.GetNumber("VZ")])
print("Before the Burn, the position and velocity are ", pos_i, vel_i)

# Apply the impulse
burn.Fire()

# Final position and velocity
pos_f = np.array([sat.GetNumber("X"), sat.GetNumber("Y"), sat.GetNumber("Z")])
vel_f = np.array([sat.GetNumber("VX"), sat.GetNumber("VY"), sat.GetNumber("VZ")])
print("After the Burn, the position and velocity are ", pos_f, vel_f)

Before the Burn, the position and velocity are  [7100.    0. 1300.] [0.   7.35 1.  ]
After the Burn, the position and velocity are  [7100.    0. 1300.] [0.         7.44908711 1.01348124]


This is an impulsive maneuver, so the position did not change.  The applied delta-V is easy to check by differencing the velocity vectors and calculating the resulting magnitude:

In [6]:
deltaVvec = vel_f - vel_i
print ("The applied delta-V is ", deltaVvec, ", with magnitude ", np.linalg.norm(deltaVvec))

The applied delta-V is  [0.         0.09908711 0.01348124] , with magnitude  0.10000000000000034


The mass depleted can also be checked.  GMAT uses the Tsiolkovsky rocket equation to calculate mass depletion for impulsive burns:

$m_f = m_1 e^{- \Delta V / v_e}$

The exhaust velocity is determined by the specific impulse applied, in seconds, and the acceleration due to gravity used for the specific impulse:

$v_e = I_{sp} g$

The impulsive burn has the settings for these values:

In [7]:
print("The applied specific impulse is ", burn.GetField("Isp"), "s and the gravitational acceleration is ", burn.GetField("GravitationalAccel"), "m/s^2")

The applied specific impulse is  300 s and the gravitational acceleration is  9.81 m/s^2


The initial mass of the configuration is 50 kg: 20 kg of fuel in a 30 kg vehicle, so the expected mass after the impulse is

In [8]:
# Apply the rocket equation, remembering that 0.1 km/s is 100 m/s
mf = 50.0 * np.exp(-100 / (300 * 9.81))
print("The expected final mass is ", mf)

# Report the spacecraft total mass after the burn
print("GMAT shows the final mass as ", sat.GetNumber("TotalMass"))

The expected final mass is  48.32959337772201
GMAT shows the final mass as  48.32959337772201


One subtlety of GMAT is the use of object clones in the system.  An example of that feature can be seen in this example.  The tank attached to the spacecraft is **a copy** of the tank that we configured above, so the original tank still has 20 kg of fuel.  The fuel mass remaining in the tank on the spacecraft is reported by retrieving the tank used in the maneuver and then accessing its data.  That tank is retrieved from the spacecraft using the GetRefObject() method.  Then, working from the retrieved fuel tank, we can access a report of the remaining fuel:

In [9]:
tankused = sat.GetRefObject(gmat.FUEL_TANK, "FuelTank")

# Finally, show the fuel usage and the fuel remaining
fuelLeft = tankused.GetNumber("FuelMass")
print((20.0 - fuelLeft), " kg of fuel was used, leaving ", fuelLeft, " kg")

1.6704066222779872  kg of fuel was used, leaving  18.329593377722013  kg
